## Lab3 - Obliczanie odległości edycyjnej

Odległość Levenshteina:

In [96]:
def get_matrix_distance(word1,word2,val = 0):
    word1,word2 = word2,word1
    n = len(word1)
    m = len(word2)
    matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]

    for i in range(n+1):
        matrix[0][i] = i
    for i in range(m+1):
        matrix[i][0] = i

    for i in range(1,m+1):
        for j in range(1,n+1):
            if(word1[j-1] == word2[i-1]):
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] = 1 + min(matrix[i-1][j-1] + val,matrix[i-1][j],matrix[i][j-1])
    return matrix

In [173]:
def get_path(word1,word2,matrix):
    w1_n = len(word1)
    w2_n = len(word2)

    oper = []
    pos = []
    while w1_n > 0 or w2_n > 0:
        if (matrix[w1_n-1][w2_n] + 1 == matrix[w1_n][w2_n]):
            oper.append("-")
            w1_n-=1
        elif(matrix[w1_n][w2_n-1] + 1 == matrix[w1_n][w2_n]):
            oper.append("+")
            w2_n-=1
        elif(matrix[w1_n-1][w2_n-1] == matrix[w1_n][w2_n]):
            oper.append("?")
            w1_n-=1
            w2_n-=1
        else:
            oper.append("/")
            w1_n-=1
            w2_n-=1
        pos.append(w2_n)
    oper.reverse()
    pos.reverse()
    return oper,pos

In [174]:
def visual(word1,word2,path):
    i = 0
    print()
    for oper in path:
        if(oper == "+"):
            print(f"Insert in {word1[:i]}[{word2[i]}]{word1[i:]}")
            word1 = word1[:i] + word2[i] + word1[i:]
            print(f"After inserting: {word1}")
            print("\n")
        elif(oper == "-"):
            print(f"Remove from {word1[:i]}[{word1[i]}]{word1[i+1:]}")
            word1 = word1[:i] + word1[i+1:]
            print(f"After removing: {word1}")
            print("\n")
            continue
        elif(oper == "/"):
            print(f"Replace from {word1[:i]}[{word1[i]} -> {word2[i]}]{word1[i+1:]}")
            word1 = word1[:i] + word2[i] + word1[i+1:]
            print(f"After replacing: {word1}")
            print("\n")
        i += 1



In [175]:
def algorithm(word1,word2):
    matrix = get_matrix_distance(word1,word2)
    print(f"Transferring from {word1} to {word2}")
    print(f"Distance: {matrix[-1][-1]}")
    path = get_path(word1,word2,matrix)[0]
    visual(word1,word2,path)


In [176]:
w1 = "los"
w2 = "kloc"
algorithm(w1,w2)


Transferring from los to kloc
Distance: 2

Insert in [k]los
After inserting: klos


Replace from klo[s -> c]
After replacing: kloc




In [177]:
w1 = "Łódź"
w2 = "Lodz"
algorithm(w1,w2)


Transferring from Łódź to Lodz
Distance: 3

Replace from [Ł -> L]ódź
After replacing: Lódź


Replace from L[ó -> o]dź
After replacing: Lodź


Replace from Lod[ź -> z]
After replacing: Lodz




In [178]:
w1 = "kwintesencja"
w2 = "quintessence"
algorithm(w1,w2)

Transferring from kwintesencja to quintessence
Distance: 5

Replace from [k -> q]wintesencja
After replacing: qwintesencja


Replace from q[w -> u]intesencja
After replacing: quintesencja


Insert in quintes[s]encja
After inserting: quintessencja


Replace from quintessenc[j -> e]a
After replacing: quintessencea


Remove from quintessence[a]
After removing: quintessence




In [179]:
w1 = "ATGAATCTTACCGCCTCG"
w2 = "ATGAGGCTCTGGCCCCTG"
algorithm(w1,w2)

Transferring from ATGAATCTTACCGCCTCG to ATGAGGCTCTGGCCCCTG
Distance: 7

Replace from ATGA[A -> G]TCTTACCGCCTCG
After replacing: ATGAGTCTTACCGCCTCG


Replace from ATGAG[T -> G]CTTACCGCCTCG
After replacing: ATGAGGCTTACCGCCTCG


Insert in ATGAGGCT[C]TACCGCCTCG
After inserting: ATGAGGCTCTACCGCCTCG


Replace from ATGAGGCTCT[A -> G]CCGCCTCG
After replacing: ATGAGGCTCTGCCGCCTCG


Insert in ATGAGGCTCTG[G]CCGCCTCG
After inserting: ATGAGGCTCTGGCCGCCTCG


Remove from ATGAGGCTCTGGCC[G]CCTCG
After removing: ATGAGGCTCTGGCCCCTCG


Remove from ATGAGGCTCTGGCCCCT[C]G
After removing: ATGAGGCTCTGGCCCCTG




Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

2 * lcs(word1,word2) = |word1| + |word2| - edid_di(word1,word2)

In [108]:
def lcs(word1,word2):
    return (len(word1) + len(word2) - get_matrix_distance(word1,word2,1)[-1][-1])/2


In [121]:
print(f"Najdłuższy wspólny podciąg: {lcs('matematyka', 'gramatyka')}")

Najdłuższy wspólny podciąg: 7.0


Korzystając z gotowego tokenizera (np spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.

Biblioteki:

In [131]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import numpy as np

In [126]:
nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)

In [127]:
with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [128]:
tokens = tokenizer(text)

Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [143]:
def del_from_token(tokens, percent=3):
    result = []
    possibility = percent/100
    for token in tokens:
        rand = np.random.random()
        if rand > possibility:
            result.append(token)
    return result


In [144]:
tokens1 = del_from_token(tokens)
tokens2 = del_from_token(tokens)

In [145]:
with open("text1.txt", "w", encoding="utf-8") as f:
    for token in tokens1:
        f.write(token.text_with_ws)

In [146]:
with open("text2.txt", "w", encoding="utf-8") as f:
    for token in tokens2:
        f.write(token.text_with_ws)

Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.

In [150]:
print(f"Najdłuższy wspólny podciąg: {lcs(tokens1,tokens2)}")

Najdłuższy wspólny podciąg: 2138.0


In [148]:
with open("text1.txt", "r", encoding="utf-8") as f:
    text1 = f.read()

In [149]:
with open("text2.txt", "r", encoding="utf-8") as f:
    text2 = f.read()

In [151]:
print(f"Najdłuższy wspólny podciąg: {lcs(text1,text2)}")


Najdłuższy wspólny podciąg: 11883.0


Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

Biblioteka:

In [153]:
from bisect import bisect

In [166]:
def lcs2(x, y):
    x_n = len(x)
    y_n = len(y)
    ranges = []
    ranges.append(y_n)

    for i in range(x_n):
        positions = [j for j in range(y_n) if y[j] == x[i]]
        positions.reverse()
        for p in positions:
            k = bisect(ranges, p)
            if k == bisect(ranges, p - 1):
                if k >= len(ranges) - 1:
                    ranges[k:k] = [p]

                else:
                    ranges[k] = p
    return len(ranges) - 1

In [167]:
print(f"Najdłuższy wspólny podciąg: {lcs2(tokens1,tokens2)}")

Najdłuższy wspólny podciąg: 2138


Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [192]:
def diff():
    with open("text1.txt", "r", encoding="utf-8") as f:
        text1 = f.read()

    with open("text2.txt", "r", encoding="utf-8") as f:
        text2 = f.read()

    text1 = text1.split('\n')
    text2 = text2.split('\n')

    matrix = get_matrix_distance(text1, text2, 1)

    operations, positions = get_path(text1, text2, matrix)

    j = 0
    for i in range(len(operations)):
        oper = operations[i]
        pos = positions[i]
        if oper == "-":
            print(f"< ({pos + j + 1}) {text2[pos + j]}")
            j += 1
        elif oper == "+":
            print(f"> ({pos + 1}) {text1[pos]}")
            j -= 1


In [194]:
diff()


> (12)  * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
> (13)  * MONTEKI, — naczelnicy dwóch domów nieprzyjaznych sobie
< (12)  * — młody Weroneńczyk szlachetnego rodu, krewny księcia
< (13)  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
> (15)  * ROMEO — syn 
< (15)  * ROMEO — syn Montekiego
> (17)  * BENWOLIO — synowiec Montekiego
< (17)  * — synowiec Montekiego
> (23)  * ABRAHAM — służący Montekiego
< (23)  * ABRAHAM — Montekiego
> (29)  * PANI MONTEKI małżonka Montekiego
< (29)  * PANI MONTEKI — małżonka Montekiego* PANI KAPULET — małżonka Kapuleta
< (30)  * JULIA — córka Kapuletów
> (32)  * MARTA — mamka Julii
< (33) 
> (41) 
< (42) 
< (44) Tam, gdzie się rzecz ta rozgrywa, w Weronie,
< (45) Do nowej zbrodni pchają złości dawne,
> (45) 
> (46) Dwa rody, zacne jednako i sławne —
< (48) Z łon tych dwu wrogów wzięło bowiem życie,
> (55) Tej ich miłości przebieg zbyt bolesny
< (57) Które otoczcie cierpliwymi względy,
< (58) Jest w nim co złego, my usu